In [264]:
import pandas as pd, numpy as np
import requests
import json
import os
import time

In [327]:
# Cufflinks wrapper on plotly
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from plotly.offline import iplot
cufflinks.go_offline()

# Set global theme

import plotly.figure_factory as ff

import plotly.graph_objects as go

# Extracting restaurant data

In [219]:
df=pd.read_excel("RE_models_input.xlsx")

In [221]:
df.district

Index(['Id', 'Area', 'Price', 'latitude', 'longitude', 'build_year',
       'building_floors_num', 'rooms_num', 'City', 'subdistrict', 'market',
       'Building_material', 'Building_ownership', 'Building_type',
       'Construction_status', 'floor_no', 'Heating', 'Windows_type',
       'Equipment_types_dishwasher', 'Equipment_types_fridge',
       'Equipment_types_furniture', 'Equipment_types_oven',
       'Equipment_types_stove', 'Equipment_types_tv',
       'Equipment_types_washing_machine', 'Extras_types_air_conditioning',
       'Extras_types_attic', 'Extras_types_balcony', 'Extras_types_basement',
       'Extras_types_garage', 'Extras_types_garden', 'Extras_types_lift',
       'Extras_types_separate_kitchen', 'Extras_types_terrace',
       'Extras_types_two_storey', 'Extras_types_usable_room',
       'Media_types_cable-television', 'Media_types_electricity',
       'Media_types_internet', 'Media_types_phone', 'Media_types_sewage',
       'Media_types_water', 'Security_types_alarm

In [239]:
df_grid=df.groupby(['lat_mod', 'lon_mod'], as_index=False).first()[['lat_mod', 'lon_mod',"district"]]

In [246]:
coors_arr=df_grid[['lat_mod', 'lon_mod']].values

In [248]:
coordinates=[]
for i in range(0, coors_list.shape[0]):
    coor_point=str(coors_arr[i,0])+", "+str(coors_arr[i,1])
    coordinates.append(coor_point)

In [210]:
coordinate = '52.1175, 21.0000'
keyword = 'restaurant'
radius=700

In [211]:
api_key = open("Api_key.txt", "r").read()

In [212]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_key)


In [365]:
coordinates[:1]

['52.0725, 21.015']

In [395]:
json_list=[]
i=0
for coordinate in coordinates[200:220]:
    i+=1
    
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location='+coordinate+'&radius='+str(radius)+'&keyword='+str(keyword)+'&key='+str(api_key)
    
    while True:

        response = requests.get(url)
        datapage = json.loads(response.text)
        json_list=json_list+datapage["results"]
        time.sleep(10)

        if 'next_page_token' not in datapage:
            break
        else:

            next_page_token = datapage['next_page_token']

            url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key='+str(api_key)+'&pagetoken='+str(next_page_token)
        
    print("Extracted {} offers from {} grids at {}".format(len(json_list),i,str(datetime.now())[:-7]))


    

Extracted 35 offers from 1 grids at 2020-06-22 20:27:04
Extracted 86 offers from 2 grids at 2020-06-22 20:27:37
Extracted 146 offers from 3 grids at 2020-06-22 20:28:11
Extracted 206 offers from 4 grids at 2020-06-22 20:28:44
Extracted 266 offers from 5 grids at 2020-06-22 20:29:18
Extracted 326 offers from 6 grids at 2020-06-22 20:29:51
Extracted 386 offers from 7 grids at 2020-06-22 20:30:25
Extracted 446 offers from 8 grids at 2020-06-22 20:30:58
Extracted 496 offers from 9 grids at 2020-06-22 20:31:31
Extracted 545 offers from 10 grids at 2020-06-22 20:32:05
Extracted 596 offers from 11 grids at 2020-06-22 20:32:38
Extracted 622 offers from 12 grids at 2020-06-22 20:33:00
Extracted 637 offers from 13 grids at 2020-06-22 20:33:11
Extracted 651 offers from 14 grids at 2020-06-22 20:33:22
Extracted 655 offers from 15 grids at 2020-06-22 20:33:33
Extracted 660 offers from 16 grids at 2020-06-22 20:33:44
Extracted 680 offers from 17 grids at 2020-06-22 20:33:55
Extracted 714 offers from

In [396]:
len(json_list)

773

## Saving extracted data to local file

In [415]:
#Saving list of jsons as a txt file

directory="./Warsaw_restaurant_data/"
if not os.path.exists(directory):
    os.makedirs(directory)
    
todaydate =   str(datetime.now())[:10]+"_"+str(datetime.now().hour)+str(datetime.now().minute)
file_name=directory+"Restaurants_"+todaydate+".txt"
file_name

'./Warsaw_restaurant_data/Restaurants_2020-06-22_2042.txt'

In [416]:
 str(datetime.now())[:10]+"_"+str(datetime.now().hour)+str(datetime.now().minute)

'2020-06-22_2042'

In [417]:
text_file = open(file_name, "w")
text_file.write(json.dumps(json_list))
text_file.close()

899868

In [418]:
#file_name='./Warsaw_restaurant_data/Restaurants_2020-06-22_2020.txt'

In [419]:
#Opening same file and converting it into a json list
json_list_txt = open(file_name, "r").read()
restaurants_json=json.loads(json_list_txt)

## Transforming data from json to dataframe

In [420]:
restaurants_json=json.dumps(json_list)
df_restaurants=pd.read_json(restaurants_json, orient='name')

df_restaurants.columns

Index(['business_status', 'geometry', 'icon', 'id', 'name', 'opening_hours',
       'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference',
       'scope', 'types', 'user_ratings_total', 'vicinity',
       'permanently_closed'],
      dtype='object')

In [421]:
df_restaurants["lat"]=df_restaurants.geometry.apply(lambda x:x["location"]["lat"])
df_restaurants["lon"]=df_restaurants.geometry.apply(lambda x:x["location"]["lng"])


In [422]:
df_restaurants.columns

Index(['business_status', 'geometry', 'icon', 'id', 'name', 'opening_hours',
       'photos', 'place_id', 'plus_code', 'price_level', 'rating', 'reference',
       'scope', 'types', 'user_ratings_total', 'vicinity',
       'permanently_closed', 'lat', 'lon'],
      dtype='object')

In [423]:
df_restaurants=df_restaurants[['name',  'place_id', 'price_level','rating', 'user_ratings_total', 'vicinity', 'lat', 'lon']]


In [424]:
df_restaurants.lat

0      52.224240
1      52.236832
2      52.227320
3      52.225870
4      52.232531
         ...    
768    52.241643
769    52.237258
770    52.238224
771    52.236516
772    52.239717
Name: lat, Length: 773, dtype: float64

In [425]:
df_restaurants=df_restaurants.drop_duplicates()

In [426]:
df_restaurants=df_restaurants[df_restaurants.lat.isna()==False]

## Visualizing extracted data 

In [427]:
#Selecting a central city point to center all graphs around - Swietokrzyska Subway 
center_coors=52.235176, 21.008393

In [433]:
data=[]
trace0=go.Scattermapbox(
            lat=df_restaurants.lat,
            lon=df_restaurants.lon,
          
            mode='markers',
            marker=dict(
                opacity=0.5, 
                size=10, 
                color=df_restaurants.rating,
               
                colorscale="jet",
                cauto=False,
                cmax=5,
                cmin=4,
                showscale=True),
                text=df_restaurants.rating,
            
           
            )



data=[trace0]

layout = go.Layout(
    title="Restaurant rating (0-5)",  
    title_x=0.4,
   
    
    
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 80, b = 0, l = 0, r = 0),
    font = dict(color = 'dark grey', size = 18),
 
    mapbox = dict(
       

        center = dict(
            lat = center_coors[0],
            lon = center_coors[1]
        ),
        # default level of zoom
        zoom = 10,
        # default map style
        style = "carto-positron"
    )

)

figure=go.Figure(
    data=data,
    layout=layout,
  
    )



iplot(figure)

In [435]:
data=[]
trace0=go.Scattermapbox(
            lat=df_restaurants.lat,
            lon=df_restaurants.lon,
          
            mode='markers',
            marker=dict(
                opacity=0.5, 
                size=10, 
                color=df_restaurants.user_ratings_total,
               
                colorscale="jet",
                cauto=False,
                cmax=1000,
                showscale=True),
                text=df_restaurants.user_ratings_total,
           
            )



data=[trace0]

layout = go.Layout(
    title="Restaurant popularity [ratings count]",  
    title_x=0.4,
   
    
    
    height = 800,
    # top, bottom, left and right margins
    margin = dict(t = 80, b = 0, l = 0, r = 0),
    font = dict(color = 'dark grey', size = 18),
 
    mapbox = dict(
       

        center = dict(
            lat = center_coors[0],
            lon = center_coors[1]
        ),
        # default level of zoom
        zoom = 10,
        # default map style
        style = "carto-positron"
    )

)

figure=go.Figure(
    data=data,
    layout=layout,
  
    )



iplot(figure)